In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import datetime
from scipy import stats

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from sklearn.linear_model import LinearRegression
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.stats import kendalltau, pearsonr, spearmanr


fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 10

ModuleNotFoundError: No module named 'numpy'

In [ ]:
SRnetworks = {}
SRnetworks['cosmos'] = pd.read_csv(r'cosmos.csv')
SRnetworks['tezos'] = pd.read_csv(r'tezos.csv')
SRnetworks['livepeer'] = pd.read_csv(r'livepeer.csv')

for net in SRnetworks:
    SRnetworks[net] = SRnetworks[net].dropna()
    SRnetworks[net]['dateTime'] = pd.to_datetime(SRnetworks[net]['timestamp'], unit='s')
    SRnetworks[net] = SRnetworks[net].set_index('dateTime')
    SRnetworks[net] = SRnetworks[net].resample('W').mean()
    plt.plot(SRnetworks[net]['total_staked'], label = net, alpha=0.9, marker = 'o', ms =3)
plt.title('Historical Staking Rate | Weekly Mean', fontsize=19)
plt.xlabel('Date', fontsize=17)
plt.ylabel('Percentage of circulating supply staked (%)', fontsize=17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc=0, fontsize=14)
plt.grid(b=True, which='major', color='#999999', linestyle='-')
plt.savefig('Hist_SR.pdf')